In [1]:
import pandas as pd

data = pd.read_csv('http://antoninofurnari.it/downloads/movieratings.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100003 entries, 0 to 100002
Data columns (total 4 columns):
user_id       100003 non-null int64
item_id       100003 non-null int64
rating        100003 non-null int64
item_title    100003 non-null object
dtypes: int64(3), object(1)
memory usage: 3.1+ MB


,user_id,item_id,rating,item_title
0,0,50,5,Star Wars (1977)
1,290,50,5,Star Wars (1977)
2,79,50,4,Star Wars (1977)
3,2,50,5,Star Wars (1977)
4,8,50,5,Star Wars (1977)


In [2]:
#Count the number of unique users and item
print("Number of unique users: {}".format(data['user_id'].nunique()))
print("Number of unique items: {}".format(data['item_id'].nunique()))

Number of unique users: 944
Number of unique items: 1682


# Domanda 1

If each user had rated each film we would have a total of 944x1682.

In the case in which every user had rated each item we would have a dense matrix. Since it is very hard that every user rate every film we have a sparse matrix

In [3]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.25)

In [4]:
# build utility matrix using a pivot table

um = train_data.pivot_table(index='user_id', columns='item_id',values='rating')
um.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1669,1672,1674,1675,1676,1677,1678,1679
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Collaborative Filtering

Considering a user who has not rated an item, build a profile for each user by considering the columns subtracting the mean, find a set of N similar users, estimate the utility value computing a weighted average of rating given by similar users

In [5]:
# Example with this values

N = 2
user = 0
item = 1

In [6]:
# we need to replace all missing values with zeros and then normalize

profiles = um.fillna(0)-um.mean(0)
profiles.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1667,1668,1669,1672,1674,1675,1676,1677,1678,1679
user_id,,,,,,,,,,,,,,,,,,,,,
0,-3.900901,-3.12766,-3.0,-3.525641,-3.220588,-3.52381,-3.808874,-4.0,-3.904545,-3.897059,...,-3.0,-3.0,-2.0,-2.0,-4.0,-3.0,-2.0,-3.0,-1.0,-3.0
1,1.099099,-0.12766,1.0,-0.525641,-0.220588,1.47619,0.191126,-4.0,-3.904545,-3.897059,...,-3.0,-3.0,-2.0,-2.0,-4.0,-3.0,-2.0,-3.0,-1.0,-3.0
2,-3.900901,-3.12766,-3.0,-3.525641,-3.220588,-3.52381,-3.808874,-4.0,-3.904545,-3.897059,...,-3.0,-3.0,-2.0,-2.0,-4.0,-3.0,-2.0,-3.0,-1.0,-3.0
3,-3.900901,-3.12766,-3.0,-3.525641,-3.220588,-3.52381,-3.808874,-4.0,-3.904545,-3.897059,...,-3.0,-3.0,-2.0,-2.0,-4.0,-3.0,-2.0,-3.0,-1.0,-3.0
4,-3.900901,-3.12766,-3.0,-3.525641,-3.220588,-3.52381,-3.808874,-4.0,-3.904545,-3.897059,...,-3.0,-3.0,-2.0,-2.0,-4.0,-3.0,-2.0,-3.0,-1.0,-3.0


# Domanda 2

We need to subtract the mean because since we are treating blank values as negative ratings, subtracting the mean will make bad ratings negative and good rating positive

In [7]:
#Now that we have calculated the user profile we need to
#compute the cosine distance between this and the item features
import numpy as np

cd = lambda x,y: np.dot(x,y)/ (np.sqrt(np.dot(x,x)) * np.sqrt(np.dot(y,y)))

profiles.apply(lambda x: cd(profiles.loc[user],x), axis=1).head()


user_id
0    1.000000
1    0.900421
2    0.978961
3    0.989802
4    0.989189
dtype: float64

In [8]:
# We need to remove the cosine distance between the same user

distances = profiles.apply(lambda x: cd(profiles.loc[user], x), axis=1).drop(user)
distances.head()

user_id
1    0.900421
2    0.978961
3    0.989802
4    0.989189
5    0.957044
dtype: float64

In [9]:
# We need now to select only the users who have rated the item

selected_users = um.loc[:,item].dropna().index
selected_users

Int64Index([  1,   5,  13,  15,  16,  18,  20,  21,  25,  26,
            ...
            919, 921, 922, 923, 924, 929, 933, 934, 938, 941],
           dtype='int64', name='user_id', length=333)

# Domanda 3

We need to select the user who have rated the item in order to average the ratings given by the k users and compute the utility value

In [10]:
distances.loc[selected_users].sort_values().tail()
#the last row is the most similar

user_id
609    0.993046
895    0.993406
202    0.993882
723    0.994091
726    0.994781
dtype: float64

In [11]:
#take the 3 most similar
selected_distances = distances.loc[selected_users].sort_values()[-3:]
selected_distances

user_id
202    0.993882
723    0.994091
726    0.994781
dtype: float64

In [12]:
#store the user ids in an array
similar_users = selected_distances.index

In [13]:
#let's see how this users have rated the item
print(um.loc[similar_users[0],item],um.loc[similar_users[1],item], um.loc[similar_users[2],item])

3.0 3.0 4.0


In [14]:
#compute average rating

predicted_rating = (um.loc[similar_users, item]*selected_distances).sum()/selected_distances.sum()

predicted_rating

3.333510983238892

In [15]:
class CollaborativeFilter():
    def __init__(self, N):
        self.N = N
    
    def fit(self, um):
        self.um = um
        self.profiles = profiles = self.um.fillna(0)-self.um.mean(0)
    
    def predict(self, user, item):
        distances = self.profiles.apply(lambda x: cd(self.profiles.loc[user], x), axis=1).drop(user)
        selected_users = self.um.loc[:,item].dropna().index
        selected_distances = distances.loc[selected_users].sort_values()[-self.N:]
        similar_users = selected_distances.index
        predicted_rating = (self.um.loc[similar_users, item]*selected_distances).sum()/selected_distances.sum()
        return predicted_rating

In [16]:
cf = CollaborativeFilter(N = 3)
cf.fit(um)
print(cf.predict(user,item))
print(cf.predict(5,18))

3.333510983238892
1.9970364039646928


In [17]:
test_data.head()

,user_id,item_id,rating,item_title
17213,451,328,5,Conspiracy Theory (1997)
59118,807,588,5,Beauty and the Beast (1991)
37783,222,215,4,Field of Dreams (1989)
82790,629,528,5,"Killing Fields, The (1984)"
6094,468,98,5,"Silence of the Lambs, The (1991)"


In [18]:
#function to iterate over the rows of test and compute ratings
from tqdm import tqdm

def predict(cf, test_data):
    predicted_ratings = []
    for i,ann in tqdm(test_data.iterrows(), total=len(test_data)):
        try:
            rating = cf.predict(ann['user_id'], ann['item_id'])
        except:
            rating = np.nan
        predicted_ratings.append(rating)
    return np.array(predicted_ratings)

In [19]:
#use a small portion of data
_, test_small = train_test_split(test_data, test_size=0.002)
len(test_small)

51

In [20]:
pr= predict(cf, test_small)

100%|██████████| 51/51 [00:32<00:00,  1.55it/s]


In [21]:
#measure performance with mae
def mae(y_true, y_pred):
    return (y_true-y_pred).abs().mean()
mae(test_small['rating'], pr)

0.8828953410668883

In [22]:
#We obtain this result since we have not many users.
cf50 = CollaborativeFilter(50)
cf50.fit(um)
pr_50 = predict(cf50, test_small)
mae(test_small['rating'], pr_50)

100%|██████████| 51/51 [00:32<00:00,  1.56it/s]


0.7868266746365926